In [ ]:
import os
import random
import json
import math
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot as plt
from glob import glob
from copy import deepcopy

import funcy
from sklearn.model_selection import train_test_split

import detectron2
import detectron2.data.datasets
import detectron2.data.catalog
import detectron2.utils.logger
import detectron2.utils.visualizer
import detectron2.config
import detectron2.engine
from detectron2.engine import DefaultPredictor
import detectron2.evaluation
import detectron2.model_zoo

from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
import detectron2.evaluation
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval


In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = [20, 10]
%reload_ext tensorboard

## Register The Dataset

In [ ]:
def save_coco(file, info, licenses, images, annotations, categories):
    with open(file, 'wt', encoding='UTF-8') as coco:
        json.dump({ 'info': info, 'licenses': licenses, 'images': images, 
            'annotations': annotations, 'categories': categories}, coco, indent=2, sort_keys=True)

def filter_annotations(annotations, images):
    image_ids = funcy.lmap(lambda i: int(i['id']), images)
    return funcy.lfilter(lambda a: int(a['image_id']) in image_ids, annotations)

def train_validate_test_split(path_to_coco_json_file, train_size=0.8, validate_size=0.2): # 0.8 train, 0.2 validate, testset is uploaded from other file
    with open(path_to_coco_json_file, 'rt', encoding='UTF-8') as annotations:
        coco = json.load(annotations)
        info = coco.get('info')
        licenses = coco.get('licenses')
        images = coco.get('images')
        annotations = coco.get('annotations')
        categories = coco.get('categories')

        number_of_images = len(images)

        images_with_annotations = funcy.lmap(lambda a: int(a['image_id']), annotations)

        images = funcy.lremove(lambda i: i['id'] not in images_with_annotations, images)
        random.shuffle(images)

        train, validate, test = np.split(images, [int(train_size*len(images)), int((train_size + validate_size)*len(images))])

        base_path = os.path.splitext(path_to_coco_json_file)[0]
        train_path = base_path + '-train.json'
        validate_path = base_path + '-validate.json'
        test_path = base_path + '-test.json'
        
        save_coco(train_path, info, licenses, list(train), filter_annotations(annotations, train), categories)
        save_coco(validate_path, info, licenses, list(validate), filter_annotations(annotations, validate), categories)
        save_coco(test_path, info, licenses, list(test), filter_annotations(annotations, test), categories)

        print("Saved {} entries in {} \nSaved {} entries in {} \nSaved {} entries in {}".format(len(train), train_path, len(validate), validate_path, len(test), test_path))
        
        return train_path, validate_path, test_path,coco

In [ ]:
# Split the dataset into train and validate sets only, test set is loaded from other file
train_file_path, validate_file_path, test_file_path,coco_return = train_validate_test_split('Custom-testest-labeled-data/Damaged_teeth_trainval-24.json')

In [ ]:
image_path = "Chipped-Tooth-Images"
detectron2.data.datasets.register_coco_instances("train", {}, train_file_path, image_path)
detectron2.data.datasets.register_coco_instances("validate", {}, validate_file_path, image_path)
detectron2.data.datasets.register_coco_instances("test", {}, 'Custom-testest-labeled-data/Damaged_teeth_testset_30_gears-23.json', image_path) # load the test set

In [ ]:
catalog_train = detectron2.data.catalog.DatasetCatalog.get("train")
metadata_train = detectron2.data.MetadataCatalog.get("train")

catalog_validate = detectron2.data.catalog.DatasetCatalog.get("validate")
metadata_validate = detectron2.data.MetadataCatalog.get("validate")

catalog_test = detectron2.data.catalog.DatasetCatalog.get("test")
metadata_test = detectron2.data.MetadataCatalog.get("test")

In [ ]:
sample = random.sample(catalog_train,1)[0]
img = cv2.imread(sample["file_name"])[:, :, ::-1] 
visualizer = detectron2.utils.visualizer.Visualizer(img, metadata_train, scale=1)
vis = visualizer.draw_dataset_dict(sample)
plt.figure()
plt.title(sample["file_name"])
plt.imshow(vis.get_image())
plt.show()

In [ ]:
cfg = detectron2.config.get_cfg()
cfg.merge_from_file(detectron2.model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("train",)
cfg.DATASETS.TEST = ("validate",)
cfg.MODEL.WEIGHTS = "detectron2://COCO-Detection/faster_rcnn_R_50_FPN_3x/137849458/model_final_280758.pkl"
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.SOLVER.IMS_PER_BATCH = 4 
cfg.SOLVER.BASE_LR = 0.01
cfg.SOLVER.GAMMA = 0.05 
cfg.SOLVER.MAX_ITER = 10000 
cfg.TEST.EVAL_PERIOD = cfg.SOLVER.MAX_ITER // 10 
cfg.OUTPUT_DIR = "./output"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

In [ ]:
class CocoTrainer(detectron2.engine.DefaultTrainer):
    
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
            os.makedirs(output_folder, exist_ok=True)

        return detectron2.evaluation.COCOEvaluator(dataset_name, cfg, False, output_folder)

In [ ]:
trainer = CocoTrainer(cfg)
trainer.resume_or_load(resume= True) # True if you want to continue from last checkpoint
trainer.train()

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DI "model_final.pth" # use it to evaluate the recent trained model
#cfg.MODEL.WEIGHTS = "Weights/New_Chipped_Tooth_Weights.pth"  # use it to loads the previously trained weights
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.75
cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0 # non-maximum suppression 
cfg.DATASETS.TEST = ("test", )
predictor = detectron2.engine.DefaultPredictor(cfg)

In [ ]:
# Inspect three random images from the test set
samples = random.sample(catalog_test, 3) 
print(len(catalog_test))
for sample in samples:
    img = cv2.imread(sample["file_name"])
    prediction = predictor(img)
    visualizer_ground_truth = detectron2.utils.visualizer.Visualizer(img[:, :, ::-1], metadata_test, scale=1)  
    visualizer_predictions = detectron2.utils.visualizer.Visualizer(img[:, :, ::-1], metadata_test, scale=1)
    #show ground truth
    vis_ground_truth = visualizer_ground_truth.draw_dataset_dict(sample)
    #show predictions
    vis_predictions = visualizer_predictions.draw_instance_predictions(prediction["instances"].to("cpu"))   
    plt.figure()
    plt.title(sample["file_name"])   
    plt.imshow(vis_ground_truth.get_image()) 
    plt.show()
    plt.figure()
    plt.title(sample["file_name"])
    plt.imshow(vis_predictions.get_image())
    plt.show()

In [ ]:
# Inspect a specific image of the test set
inspected_image = cv2.imread('path-to-image')
predicted_instances = predictor(inspected_image)
visualizer_predictions_inspected = detectron2.utils.visualizer.Visualizer(inspected_image[:, :, ::-1], metadata_test, scale=1)
vis_predictions_inspected = visualizer_predictions_inspected.draw_instance_predictions(predicted_instances["instances"].to("cpu"))
print(predicted_instances["instances"].get('pred_boxes'))
plt.imshow(vis_predictions_inspected.get_image())

plt.show()
    

## Run these cells to plot the precision and recall values at specific thresholds (.75 to .95)

In [ ]:
def report_metrics(threshold_for_json):
    """Exports two json file which contain the ground truth and predictions bboxes and image's name in a certain format

    Args:
        Different thresholds

    Returns:
        saves two json files, one for ground truth boxes and the other for prediction boxes

    """
    predictions_dict ={}
    ground_truth_dict = {}
    false_positives_images = []
    false_negatives_images = []
    total_ground_truth = 0
    true_defects_number = 0
    for example in catalog_test :
        annotations_array=[]      
        img = cv2.imread(example["file_name"])
        prediction = predictor(img)
        
        
        visualizer_ground_truth = detectron2.utils.visualizer.Visualizer(img[:, :, ::-1], metadata_test, scale=1)
        visualizer_predictions = detectron2.utils.visualizer.Visualizer(img[:, :, ::-1], metadata_test, scale=1)
        vis_ground_truth = visualizer_ground_truth.draw_dataset_dict(example)        
        vis_predictions = visualizer_predictions.draw_instance_predictions(prediction["instances"].to("cpu"))
        
        # uncomment the following lines to show ground truth
        
#         plt.figure() 
#         plt.title(example["file_name"]) 
#         plt.imshow(vis_ground_truth.get_image()) 
#         plt.show() 
        
        # uncomment the following lines to show predictions
        
#         plt.figure() 
#         plt.title(example["file_name"]) 
#         plt.imshow(vis_predictions.get_image()) 
#         plt.show() 
        
        # Getting bboxes for ground truth
        total_ground_truth = total_ground_truth + len(example['annotations'])
        for anno in range (len(example['annotations'])):
            bounding_coord = example['annotations'][anno]['bbox']

            modified_boxes = [bounding_coord[0],bounding_coord[1],bounding_coord[0]+bounding_coord[2],bounding_coord[1]+bounding_coord[3]]

            annotations_array.append(modified_boxes)
       
        ground_dict= {
          os.path.basename(example["file_name"]):annotations_array
        }
        ground_truth_dict.update(ground_dict) 
   
        with open("ground_truth_chipped_all"+str(threshold_for_json)+".json", "w") as outfile: 
            json.dump(ground_truth_dict, outfile)


        predictions_boxes = prediction["instances"].get('pred_boxes')
        predictions_scores = prediction["instances"].get('scores')
        boxes_array=[]
        scores_array=[]

        # Getting bboxes and scores for predicitons

        for predict in range(len(predictions_boxes)):
            coordinates = [round(float(list(predictions_boxes)[predict][0]),4),round(float(list(predictions_boxes)[predict][1]),4),round(float(list(predictions_boxes)[predict][2]),4),round(float(list(predictions_boxes)[predict][3]),4)]
            scores = round(float(predictions_scores[predict]),4)
            boxes_array.append(coordinates)
            scores_array.append(scores)
        pred_dict= {
          os.path.basename(example["file_name"]):{
          "boxes": boxes_array,
          "scores": scores_array
        }}
        predictions_dict.update(pred_dict)        
        with open("predictions_chipped_all"+str(threshold_for_json)+".json", "w") as outfile: 
            json.dump(predictions_dict, outfile) 


"""
author: Timothy C. Arlen
date: 28 Feb 2018

edited by : Abdelrahman Allam
"""

COLORS = [
    '#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', '#2ca02c',
    '#98df8a', '#d62728', '#ff9896', '#9467bd', '#c5b0d5',
    '#8c564b', '#c49c94', '#e377c2', '#f7b6d2', '#7f7f7f',
    '#c7c7c7', '#bcbd22', '#dbdb8d', '#17becf', '#9edae5']


def calc_iou_individual(pred_box, gt_box):
    """Calculate IoU of single predicted and ground truth box

    Args:
        pred_box (list of floats): location of predicted object as
            [xmin, ymin, xmax, ymax]
        gt_box (list of floats): location of ground truth object as
            [xmin, ymin, xmax, ymax]

    Returns:
        float: value of the IoU for the two boxes.

    Raises:
        AssertionError: if the box is obviously malformed
    """
    x1_t, y1_t, x2_t, y2_t = gt_box
    x1_p, y1_p, x2_p, y2_p = pred_box

    if (x1_p > x2_p) or (y1_p > y2_p):
        raise AssertionError(
            "Prediction box is malformed? pred box: {}".format(pred_box))
    if (x1_t > x2_t) or (y1_t > y2_t):
        raise AssertionError(
            "Ground Truth box is malformed? true box: {}".format(gt_box))

    if (x2_t < x1_p or x2_p < x1_t or y2_t < y1_p or y2_p < y1_t):
        return 0.0

    far_x = np.min([x2_t, x2_p])
    near_x = np.max([x1_t, x1_p])
    far_y = np.min([y2_t, y2_p])
    near_y = np.max([y1_t, y1_p])

    inter_area = (far_x - near_x + 1) * (far_y - near_y + 1)
    true_box_area = (x2_t - x1_t + 1) * (y2_t - y1_t + 1)
    pred_box_area = (x2_p - x1_p + 1) * (y2_p - y1_p + 1)
    iou = inter_area / (true_box_area + pred_box_area - inter_area)
    return iou


def get_single_image_results(gt_boxes, pred_boxes, iou_thr):
    """Calculates number of true_pos, false_pos, false_neg from single batch of boxes.

    Args:
        gt_boxes (list of list of floats): list of locations of ground truth
            objects as [xmin, ymin, xmax, ymax]
        pred_boxes (dict): dict of dicts of 'boxes' (formatted like `gt_boxes`)
            and 'scores'
        iou_thr (float): value of IoU to consider as threshold for a
            true prediction.

    Returns:
        dict: true positives (int), false positives (int), false negatives (int)
    """

    all_pred_indices = range(len(pred_boxes))
    all_gt_indices = range(len(gt_boxes))
    if len(all_pred_indices) == 0:
        tp = 0
        fp = 0
        fn = len(gt_boxes)
        return {'true_pos': tp, 'false_pos': fp, 'false_neg': fn}
    if len(all_gt_indices) == 0:
        tp = 0
        fp = len(pred_boxes)
        fn = 0
        return {'true_pos': tp, 'false_pos': fp, 'false_neg': fn}

    gt_idx_thr = []
    pred_idx_thr = []
    ious = []
    for ipb, pred_box in enumerate(pred_boxes):
        for igb, gt_box in enumerate(gt_boxes):
            iou = calc_iou_individual(pred_box, gt_box)
            if iou > iou_thr:
                gt_idx_thr.append(igb)
                pred_idx_thr.append(ipb)
                ious.append(iou)

    args_desc = np.argsort(ious)[::-1]
    if len(args_desc) == 0:
        # No matches
        tp = 0
        fp = len(pred_boxes)
        fn = len(gt_boxes)
    else:
        gt_match_idx = []
        pred_match_idx = []
        for idx in args_desc:
            gt_idx = gt_idx_thr[idx]
            pr_idx = pred_idx_thr[idx]
            # If the boxes are unmatched, add them to matches
            if (gt_idx not in gt_match_idx) and (pr_idx not in pred_match_idx):
                gt_match_idx.append(gt_idx)
                pred_match_idx.append(pr_idx)
        tp = len(gt_match_idx)
        fp = len(pred_boxes) - len(pred_match_idx)
        fn = len(gt_boxes) - len(gt_match_idx)

    return {'true_pos': tp, 'false_pos': fp, 'false_neg': fn}


def calc_precision_recall(img_results):
    """Calculates precision and recall from the set of images

    Args:
        img_results (dict): dictionary formatted like:
            {
                'img_id1': {'true_pos': int, 'false_pos': int, 'false_neg': int},
                'img_id2': ...
                ...
            }

    Returns:
        tuple: of floats of (precision, recall)
    """
    true_pos = 0; false_pos = 0; false_neg = 0   
    for _, res in img_results.items():
        true_pos += res['true_pos']
        false_pos += res['false_pos']
        false_neg += res['false_neg']
    try:
        precision = true_pos/(true_pos + false_pos)
    except ZeroDivisionError:
        precision = 0.0
    try:
        recall = true_pos/(true_pos + false_neg)
    except ZeroDivisionError:
        recall = 0.0

    return (precision, recall,true_pos,false_pos,false_neg)

def get_model_scores_map(pred_boxes):
    """Creates a dictionary of from model_scores to image ids.

    Args:
        pred_boxes (dict): dict of dicts of 'boxes' and 'scores'

    Returns:
        dict: keys are model_scores and values are image ids (usually filenames)

    """
    model_scores_map = {}
    for img_id, val in pred_boxes.items():
        for score in val['scores']:
            if score not in model_scores_map.keys():
                model_scores_map[score] = [img_id]
            else:
                model_scores_map[score].append(img_id)
    return model_scores_map

def get_avg_precision_at_iou(gt_boxes, pred_boxes, iou_thr=0.5):
    """Calculates average precision at given IoU threshold.

    Args:
        gt_boxes (list of list of floats): list of locations of ground truth
            objects as [xmin, ymin, xmax, ymax]
        pred_boxes (list of list of floats): list of locations of predicted
            objects as [xmin, ymin, xmax, ymax]
        iou_thr (float): value of IoU to consider as threshold for a
            true prediction.

    Returns:
        dict: avg precision as well as summary info about the PR curve

        Keys:
            'avg_prec' (float): average precision for this IoU threshold
            'precisions' (list of floats): precision value for the given
                model_threshold
            'recall' (list of floats): recall value for given
                model_threshold
            'models_thrs' (list of floats): model threshold value that
                precision and recall were computed for.
    """
    model_scores_map = get_model_scores_map(pred_boxes)
    sorted_model_scores = sorted(model_scores_map.keys())
    # Sort the predicted boxes in descending order (lowest scoring boxes first):
    for img_id in pred_boxes.keys():
        arg_sort = np.argsort(pred_boxes[img_id]['scores'])
        pred_boxes[img_id]['scores'] = np.array(pred_boxes[img_id]['scores'])[arg_sort].tolist()
        pred_boxes[img_id]['boxes'] = np.array(pred_boxes[img_id]['boxes'])[arg_sort].tolist()

    pred_boxes_pruned = deepcopy(pred_boxes)
    precisions = []
    recalls = []
    model_thrs = []
    img_results = {}
    false_positives_images_names = [] 
    total_gears_number = []
    number_false_positives_per_image = []
    # Loop over model score thresholds and calculate precision, recall    
    for ithr, model_score_thr in enumerate(sorted_model_scores[:-1]):
        # On first iteration, define img_results for the first time:
        img_ids = gt_boxes.keys() if ithr == 0 else model_scores_map[model_score_thr]
        for img_id in img_ids:
            gt_boxes_img = gt_boxes[img_id]
            box_scores = pred_boxes_pruned[img_id]['scores']
            start_idx = 0
            for score in box_scores:
                if score <= model_score_thr:
                    pred_boxes_pruned[img_id]
                    start_idx += 1
                else:
                    break

            # Remove boxes, scores of lower than threshold scores:
            pred_boxes_pruned[img_id]['scores'] = pred_boxes_pruned[img_id]['scores'][start_idx:]
            pred_boxes_pruned[img_id]['boxes'] = pred_boxes_pruned[img_id]['boxes'][start_idx:]
            # Recalculate image results for this image
            img_results[img_id] = get_single_image_results(
                gt_boxes_img, pred_boxes_pruned[img_id]['boxes'], iou_thr)

            total_gears_number.append(img_id.split('_')[0]) 

            if img_results[img_id]['false_pos'] > 0:
                false_positives_images_names.append(img_id.split('_')[0])
                number_false_positives_per_image.append(img_results[img_id]['false_pos'])

        predictions_numbers_dict= pd.DataFrame({
        'gear_number':false_positives_images_names,
        "predictions_number": number_false_positives_per_image,
    
        })
        predictions_numbers_dict= predictions_numbers_dict.set_index('gear_number')   
        prec, rec,total_tp,total_fp,total_fn = calc_precision_recall(img_results)
        prec = prec * 100
        prec = np.round(prec)
        rec = rec * 100
        rec = np.round(rec)
        precisions.append(prec)
        recalls.append(rec)
        model_thrs.append(model_score_thr)
     
        #break 


    precisions = np.array(precisions)
    recalls = np.array(recalls)
    


    prec_at_rec = []
    for recall_level in np.linspace(0.0, 1.0, 11):
        try:
            args = np.argwhere(recalls >= recall_level).flatten()
            prec = max(precisions[args])
        except ValueError:
            prec = 0.0
        prec_at_rec.append(prec)

    avg_prec = np.mean(prec_at_rec)
    return { 'Threshold':model_score_thr,
             'True_positives': total_tp,
             'False_positives': total_fp,
             'False_negatives': total_fn,
             'Precision': prec,
             'Recall': rec,
             'avg_prec': avg_prec,
             'precisions': precisions,
             'recalls': recalls,
             'model_thrs': model_thrs}


In [ ]:
# Inspection of the test set images with different thresholds
different_thresholds = [.75,.8,.85,.9,.95]
print('catalog_test images number',len(catalog_test)) # number of test images
precisions_for_chart = []
recalls_for_chart = []
for threshold_for_json in different_thresholds:
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = threshold_for_json  # Sets threshold for this model
    predictor = detectron2.engine.DefaultPredictor(cfg)
    report_metrics(threshold_for_json)      
    with open('ground_truth_chipped_all'+str(threshold_for_json)+'.json') as infile:
        gt_boxes = json.load(infile)   
    with open('predictions_chipped_all'+str(threshold_for_json)+'.json') as infile:
        pred_boxes = json.load(infile)
    iou_thr = 0.5    
    data = get_avg_precision_at_iou(gt_boxes, pred_boxes, iou_thr=iou_thr)
    precisions_for_chart.append(data['Precision'])
    recalls_for_chart.append(data['Recall'])
    print('Metrics for threshold '+str(threshold_for_json)+' are', data)
print(precisions_for_chart)
print(recalls_for_chart)

In [ ]:
# Plotting bar chart for precision and recall with different thresholds
N = 5 
precision = precisions_for_chart
recall = recalls_for_chart
fig, ax = plt.subplots()
ind = np.arange(N) 
width = 0.15       
precision_bar = ax.bar(ind, precision, width, label='Precision')
recall_bar = ax.bar(ind + width, recall, width,
    label='Recall')


def autolabel(rects):
    """
    Attach a text label above each bar displaying its height
    """
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2., 1*height,
                '%d' % int(height),
                ha='center', va='bottom',fontsize=20)
autolabel(precision_bar)
autolabel(recall_bar)       

# for i, v in enumerate(precision):
#     ax.text(v + 3, i + .25, str(v), color='blue', fontweight='bold')
    
ax.set_xlabel('Threshold',fontsize=20)
ax.set_ylabel('Percentage',fontsize=20)
ax.set_title('Precision and recall of 300 images',fontsize=20)


plt.xticks(ind + width / 2, ('75%', '80%', '85%', '90%', '95%'),fontsize=20)
plt.legend(loc='upper right',prop={"size":10})
plt.show()

## Run these cells to plot the precision and recall curve

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0 # set to 0 to plot precision and recall values at all thresholds
cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0 # non-maximum suppression 

cfg.DATASETS.TEST = ("test", )
#predictor = DefaultPredictor(cfg)

predictor = detectron2.engine.DefaultPredictor(cfg)

In [ ]:
predictions_dict ={}
ground_truth_dict = {}
false_positives_images = []
false_negatives_images = []
total_ground_truth = 0
true_defects_number = 0
print('catalog_test images number',len(catalog_test))
for example in catalog_test :
    
    annotations_array=[]
    img = cv2.imread(example["file_name"])
    prediction = predictor(img)
    visualizer_ground_truth = detectron2.utils.visualizer.Visualizer(img[:, :, ::-1], metadata_test, scale=1)  
    visualizer_predictions = detectron2.utils.visualizer.Visualizer(img[:, :, ::-1], metadata_test, scale=1)
    #show ground truth
    vis_ground_truth = visualizer_ground_truth.draw_dataset_dict(example)
    #show predictions
    vis_predictions = visualizer_predictions.draw_instance_predictions(prediction["instances"].to("cpu"))   
    
    # Getting bboxes for ground truth
    
    total_ground_truth = total_ground_truth + len(example['annotations'])
    for anno in range (len(example['annotations'])):
        bounding_coord = example['annotations'][anno]['bbox']
        
        modified_boxes = [bounding_coord[0],bounding_coord[1],bounding_coord[0]+bounding_coord[2],bounding_coord[1]+bounding_coord[3]]
        annotations_array.append(modified_boxes)
    ground_dict= {
      os.path.basename(example["file_name"]):annotations_array
    }
    ground_truth_dict.update(ground_dict) 
    with open("ground_truth.json", "w") as outfile: 
        json.dump(ground_truth_dict, outfile)
    
   
    predictions_boxes = prediction["instances"].get('pred_boxes')
    predictions_scores = prediction["instances"].get('scores')
    boxes_array=[]
    scores_array=[]
    
    # Getting bboxes and scores for predicitons
    
    for predict in range(len(predictions_boxes)):

        coordinates = [round(float(list(predictions_boxes)[predict][0]),4),round(float(list(predictions_boxes)[predict][1]),4),round(float(list(predictions_boxes)[predict][2]),4),round(float(list(predictions_boxes)[predict][3]),4)]      
        scores = round(float(predictions_scores[predict]),4)     
        boxes_array.append(coordinates)
        scores_array.append(scores)
    pred_dict= {
      os.path.basename(example["file_name"]):{
      "boxes": boxes_array,
      "scores": scores_array
    }}   
    predictions_dict.update(pred_dict) 
    with open("predictions.json", "w") as outfile: 
        json.dump(predictions_dict, outfile) 
print("DONE")        

In [ ]:
def get_avg_precision_at_iou(gt_boxes, pred_boxes, iou_thr=0.5):
    """Calculates average precision at given IoU threshold.

    Args:
        gt_boxes (list of list of floats): list of locations of ground truth
            objects as [xmin, ymin, xmax, ymax]
        pred_boxes (list of list of floats): list of locations of predicted
            objects as [xmin, ymin, xmax, ymax]
        iou_thr (float): value of IoU to consider as threshold for a
            true prediction.

    Returns:
        dict: avg precision as well as summary info about the PR curve

        Keys:
            'avg_prec' (float): average precision for this IoU threshold
            'precisions' (list of floats): precision value for the given
                model_threshold
            'recall' (list of floats): recall value for given
                model_threshold
            'models_thrs' (list of floats): model threshold value that
                precision and recall were computed for.
    """

    model_scores_map = get_model_scores_map(pred_boxes)
    sorted_model_scores = sorted(model_scores_map.keys())
    

    # Sort the predicted boxes in descending order (lowest scoring boxes first):
    for img_id in pred_boxes.keys():
        arg_sort = np.argsort(pred_boxes[img_id]['scores'])
        pred_boxes[img_id]['scores'] = np.array(pred_boxes[img_id]['scores'])[arg_sort].tolist()
        pred_boxes[img_id]['boxes'] = np.array(pred_boxes[img_id]['boxes'])[arg_sort].tolist()

    pred_boxes_pruned = deepcopy(pred_boxes)

    precisions = []
    recalls = []
    model_thrs = []
    img_results = {}
    false_positives_images_names = [] 
    total_gears_number = []
    number_false_positives_per_image = []
    # Loop over model score thresholds and calculate precision, recall
    
   
    for ithr, model_score_thr in enumerate(sorted_model_scores[:-1]):
        # On first iteration, define img_results for the first time:
        img_ids = gt_boxes.keys() if ithr == 0 else model_scores_map[model_score_thr]
        for img_id in img_ids:
            gt_boxes_img = gt_boxes[img_id]
            box_scores = pred_boxes_pruned[img_id]['scores']
            start_idx = 0
            for score in box_scores:
                if score <= model_score_thr:
                    pred_boxes_pruned[img_id]
                    start_idx += 1
                else:
                    break

            # Remove boxes, scores of lower than threshold scores:
            pred_boxes_pruned[img_id]['scores'] = pred_boxes_pruned[img_id]['scores'][start_idx:]
            pred_boxes_pruned[img_id]['boxes'] = pred_boxes_pruned[img_id]['boxes'][start_idx:]

            
            img_results[img_id] = get_single_image_results(
                gt_boxes_img, pred_boxes_pruned[img_id]['boxes'], iou_thr)

            total_gears_number.append(img_id.split('_')[0]) # I added this to get the total number of gears

            if img_results[img_id]['false_pos'] > 0:
                false_positives_images_names.append(img_id.split('_')[0])
                number_false_positives_per_image.append(img_results[img_id]['false_pos'])

       
        predictions_numbers_dict= pd.DataFrame({
        'gear_number':false_positives_images_names,
        "predictions_number": number_false_positives_per_image,
    
        })
        predictions_numbers_dict= predictions_numbers_dict.set_index('gear_number')

        prec, rec,total_tp,total_fp,total_fn = calc_precision_recall(img_results)



        precisions.append(prec)
        recalls.append(rec)

        model_thrs.append(model_score_thr)

    precisions = np.array(precisions)
    recalls = np.array(recalls)

    prec_at_rec = []
    for recall_level in np.linspace(0.0, 1.0, 11):
        try:
            args = np.argwhere(recalls >= recall_level).flatten()
            prec = max(precisions[args])
        except ValueError:
            prec = 0.0
        prec_at_rec.append(prec)

    avg_prec = np.mean(prec_at_rec)

    return { 'Threshold':model_score_thr,
             'True_positives': total_tp,
             'False_positives': total_fp,
             'False_negatives': total_fn,
             'Precision': prec,
             'Recall': rec,
             'avg_prec': avg_prec,
             'precisions': precisions,
             'recalls': recalls,
             'model_thrs': model_thrs}


def plot_pr_curve(
    precisions, recalls, category='Chipped-Tooth', label=None, color=None, ax=None):
    """Simple plotting helper function"""

    if ax is None:
        plt.figure(figsize=(10,8))
        ax = plt.gca()

    if color is None:
        color = COLORS[0]
    ax.scatter(recalls, precisions, label=label, s=5, color='#1f77b4') 
    ax.set_xlabel('Recall')
    ax.set_ylabel('Precision')

    ax.set_title('Precision-Recall curve of Chipped-Tooth Defect (306 images)')
    #ax.set_title('Precision-Recall curve of damaged teeth 10th fold')
    ax.set_xlim([0.0,1.1])
    ax.set_ylim([0.0,1.1])
    return ax


if __name__ == "__main__":

    with open('ground_truth.json') as infile:
        gt_boxes = json.load(infile)

    with open('predictions.json') as infile:
        pred_boxes = json.load(infile)

    # Runs it for one IoU threshold
    iou_thr = 0.5 
   
    data = get_avg_precision_at_iou(gt_boxes, pred_boxes, iou_thr=iou_thr)
    ax = None
    avg_precs = []
    iou_thrs = []
    recalls_printed = []

   
    avg_precs.append(data['avg_prec'])
    iou_thrs.append(iou_thr)


    precisions = data['precisions']
    recalls = data['recalls']

    
    for idx, iou_thr in enumerate(np.linspace(0.5, 0.5,10)):

        data = get_avg_precision_at_iou(gt_boxes, pred_boxes, iou_thr=iou_thr)
        avg_precs.append(data['avg_prec'])
        iou_thrs.append(iou_thr)
        precisions = data['precisions']
        recalls = data['recalls']
        ax = plot_pr_curve(
            precisions, recalls, label='{:.2f}'.format(iou_thr), color=COLORS[idx*2], ax=ax)

    # prettify for printing:
    avg_precs = [float('{:.4f}'.format(ap)) for ap in avg_precs]
    iou_thrs = [float('{:.4f}'.format(thr)) for thr in iou_thrs]

   


    for xval in np.linspace(0.0, 1.0, 11):
        plt.vlines(xval, 0.0, 1.1, color='gray', alpha=0.3, linestyles='dashed')
        plt.hlines(xval, 0.0, 1.1, color='gray', alpha=0.3, linestyles='dashed')

    
    plt.show()